In [3]:
import numpy as np      
import matplotlib.pyplot as plt 
import scipy.io.wavfile 
import subprocess
import librosa
import librosa.display
import IPython.display as ipd
import os
import pickle
from collections import defaultdict, Counter
import re

from pathlib import Path, PurePath   
from tqdm import tqdm

## Settings

In [4]:
N_TRACKS = 1413
N_QUERYS = 10
HOP_SIZE = 512
OFFSET = 0
DURATION = 600
THRESHOLD = 0
BAND_SIZE = 10
PERMUTATIONS = 300
SEED = 26
MOD = 1423

In [5]:
#path variables
data_folder = Path("./mp3s-32k/")
mp3_tracks = data_folder.glob("*/*/*.mp3")
tracks = data_folder.glob("*/*/*.wav")

# Utility functions

In [6]:
def convert_mp3_to_wav(audio:str) -> str:  
    """Convert an input MP3 audio track into a WAV file.

    Args:
        audio (str): An input audio track.

    Returns:
        [str]: WAV filename.Spero che siate aggiornati sulla stagione UK???? ￼￼￼￼￼
    """
    if audio[-3:] == "mp3":
        wav_audio = audio[:-3] + "wav"
        if not Path(wav_audio).exists():
                subprocess.check_output(f"ffmpeg -i {audio} {wav_audio}", shell=True)
        return wav_audio
    
    return audio

def plot_spectrogram_and_peaks(track:np.ndarray, sr:int, peaks:np.ndarray, onset_env:np.ndarray) -> None:
    """Plots the spectrogram and peaks 

    Args:
        track (np.ndarray): A track.
        sr (int): Aampling rate.
        peaks (np.ndarray): Indices of peaks in the track.
        onset_env (np.ndarray): Vector containing the onset strength envelope.
    """
    times = librosa.frames_to_time(np.arange(len(onset_env)),
                            sr=sr, hop_length=HOP_SIZE)

    plt.figure(figsize=(15,10))
    ax = plt.subplot(2, 1, 2)
    D = librosa.stft(track)
    librosa.display.specshow(librosa.amplitude_to_db(np.abs(D), ref=np.max),
                            y_axis='log', x_axis='time')
    plt.subplot(2, 1, 1, sharex=ax)
    plt.plot(times, onset_env, alpha=0.8, label='Onset strength')
    plt.vlines(times[peaks], 0,
            onset_env.max(), color='r', alpha=0.8,
            label='Selected peaks')
    plt.legend(frameon=True, framealpha=0.8)
    plt.axis('tight')
    plt.tight_layout()
    plt.show()

def load_audio_peaks(audio, offset, duration, hop_size):
    """Load the tracks and peaks of an audio.

    Args:
        audio (string, int, pathlib.Path or file-like object): [description]
        offset (float): start reading after this time (in seconds)
        duration (float): only load up to this much audio (in seconds)
        hop_size (int): the hop_length

    Returns:
        tuple: Returns the audio time series (track) and sampling rate (sr), 
        a vector containing the onset strength envelope
        (onset_env), and the indices of peaks in track (peaks).
    """
    try:
        track, sr = librosa.load(audio, offset=offset, duration=duration)
        onset_env = librosa.onset.onset_strength(track, sr=sr, hop_length=hop_size)
        peaks = librosa.util.peak_pick(onset_env, 10, 10, 10, 10, 0.5, 0.5)
    except Error as e:
        print('An error occurred processing ', str(audio))
        print(e)

    return track, sr, onset_env, peaks
    
    

# Minhash

## Building the signatures of the tracks

Computing longest track duration:

In [7]:
#computing the track duration from the first element of the dataset
tracks = sorted(data_folder.glob("*/*/*.wav"), key=os.path.getsize) # sorting tracks by size
_, sr, onset_env, _ = load_audio_peaks(tracks[-1], OFFSET, DURATION, HOP_SIZE) #input longest track
TrackDuration = librosa.frames_to_time(np.arange(len(onset_env)), sr=sr, hop_length=HOP_SIZE) #max size track duration

The method used to compute the minhash value of each track is:
1) Extraxt audio peaks
2) A  ShingleArray is build, it represents the track only with it's peaks:
    - The track is an array of zeros and ones 
    - Each array element represents a given moment in time
    - Ones represent peaks, and so the elements of the array are all zeros except at the moment in which a peak was computed
3) The elements of the array are randomly permuted #PERMUTATOINS times; the seed of the permuting algorithm is initialized for each track
4) At each permutation the index of the first non zero element is recorded in the SignatureArray of the track to guarantee conformity
5) Each track is assigned an ID, the correspondance ID - track_path is recorded in a dictionary; this dictionay is dumped to a file to be recalled without re-running the code 
6) The ShingleArray and the SignatureArray are stored in a the ShingleMatrix and the SignatureMatrix with the columns corresponding to the track ID; this matrixes are dumped to a file to be recalled without re-running the code 

In [52]:
#redefining tracks to re-start from the first element of tracks generator
tracks = data_folder.glob("*/*/*.wav")

#dictionary to associate track ID row with it's path
IDofTrack = dict()

#defining the SignatureArray
SignatureMatrix = np.empty((PERMUTATIONS, N_TRACKS), dtype=int) #row index: permutation number, column index: ID
ShingleMatrix = np.empty((len(TrackDuration), N_TRACKS), dtype=int) #row: time instant, column index: ID

#buiding the shingle matrix and the first row of the signature matrix
for ID, audio in enumerate(tqdm(tracks, total=N_TRACKS)):
    AudioPath = '/'.join(os.path.abspath(audio).split('/')[-3:]) #formatting the path to have only essential info
    IDofTrack[ID] = AudioPath
    _, _, _, peaks = load_audio_peaks(audio, OFFSET, DURATION, HOP_SIZE) #computing peak indexes
    
    #buiding a Shingle array
    ShingleArray =  np.zeros(len(TrackDuration), dtype=int)
    for PeakIdx in peaks:
        ShingleArray[PeakIdx] = 1
    ShingleMatrix[:, ID] = ShingleArray
    
    #permuting the shingle array and recording it's the first non null element in the SignatureArray
    #initializing the random number generator to obtain consistent permutations across all tracks
    rng = np.random.default_rng(SEED) 
    for i in range(PERMUTATIONS):
        ShuffledShingleArray = rng.permutation(ShingleArray)
        SignatureMatrix[i][ID] = ShuffledShingleArray.argmax()
        
#storing the computed matrixes
with open('ShingleMatrix', 'wb') as f:
    pickle.dump(ShingleMatrix, f)
with open('SignatureMatrix', 'wb') as f:
    pickle.dump(SignatureMatrix, f)
with open('IDofTrack', 'wb') as f:
    pickle.dump(IDofTrack, f)

Importing SignatureMatrix and IDofTracks instead of running minhash code:

In [8]:
pickle_SignatureMatrix = open('SignatureMatrix', "rb")
SignatureMatrix = pickle.load(pickle_SignatureMatrix)

pickle_IDofTrack = open('IDofTrack', "rb")
IDofTrack = pickle.load(pickle_IDofTrack)

## Hashing

The algorithm below is a basic hashing function. 
1) The elements of the input array are multiplied to a random number
2) They are then summed
3) The output is the reminder of the division by MOD

MOD is the closest prime number to N_TRACKS, this condition limits random collisions, i.e. the hashed elements will be almost equally distributed through the #MOD bins

In [9]:
def HashFunction(Shingle):
    np.random.seed(SEED)
    c = np.array([np.random.randint(MOD) for i in range(BAND_SIZE)])
    return np.multiply(Shingle, c).sum() % MOD

A bag method is used for bucketing:
1) The SignatureArray of each track is divided in to arrays of lenght BAND_SIZE; we refer to those array as tokens, a track is now represented through the set of it's tokens
2) The hash values of this tokens are computed through the hash function
3) A dictionary is build:
    - it's keys are the hash value of a given token
    - the values of the keys are the buckets containing the track ID's in which have the corresponding token
    - this values are counters that store the number of occurences of the token of each track

In [10]:
def BucketingShingles():
    ShingleBuckets = defaultdict(Counter)
    #hashing the audio track using as keys it's signatures grouped in bands of size BAND_SIZE
    for ID in range(N_TRACKS):
        for i in range(0, PERMUTATIONS, BAND_SIZE):   
            Shingle = SignatureMatrix[:, ID][i:(i+BAND_SIZE)]
            BucketKey = HashFunction(Shingle)
            ShingleBuckets[str(BucketKey)][ID] +=1
    return(ShingleBuckets)

# Querying

The minhash method was used to process the querys in the same manner as the dataset.
<br>
<br>

For each token of the query the corresponding bucket is retrived. It is remainded that this bucket contains the tracks that contained the token, associated with the number of occurences of that token.\
A TrackSimilarities counter is now used to store the number of occurences of the tracks of all the query tokens. This method de facto groups the tracks and it's number of occurences contained in all the buckets to which the query belongs.\
The tracks with most occurances are then retrieved from the TrackSimilarities dictionary to make a comparision of the respective SignatureArrays with the query. The number of similarities divided by the number of array elements will be the Jaccard Similiraity of the tracks.

As defined in Mining of Massive Datasets by Jure Leskovec the threshold $T$ is the value of similarity $JS$ at which the probability of becoming a candidate is $1/2$. Conforming to the book notation:
- $b$ = `#BANDS`
- $r$ = `BAND_SIZE`

with `#BANDS = PERMUTATIONS / BAND_SIZE` that is:
- $b$ = `PERMUTATIONS` / $r$

An approximation to the threshold is $(\frac{1}{r})^{1 / b}$.
<br>
<br>

The output of the following function `EvaluateQuerys()` code cell prints:
- the query track itself. This was checked by a direct audio comparison of the audio of the query and the best match
- the second best match with it's Jaccard Similarity score

In [11]:
def EvaluateQuerys():
    #defing path variables
    query_folder = Path("./querys")
    querys = sorted(query_folder.glob("*.wav"), key=os.path.getmtime)

    #buiding the shingle matrix and the first row of the signature matrix
    for ID, audio in enumerate(querys):
        # if ID < 1:
        AudioPath = os.path.abspath(audio).split('/')[-1] #formatting the path to have only essential info
        _, _, _, peaks = load_audio_peaks(audio, OFFSET, DURATION, HOP_SIZE) #computing peack indexes

        #buiding a Shingle array
        ShingleArray =  np.zeros(len(TrackDuration), dtype=int)
        for PeakIdx in peaks:
          ShingleArray[PeakIdx] = 1

        #defining the SignatureArray
        SignatureArray = np.empty(PERMUTATIONS, dtype=int)

        #permuting the shingle array and recording it's the first non null element in the SignatureArray
        #initializing the random number generator to obtain consistent permutations across all tracks
        rng = np.random.default_rng(SEED) 
        for i in range(PERMUTATIONS):
            ShuffledShingleArray = rng.permutation(ShingleArray)
            SignatureArray[i] = ShuffledShingleArray.argmax()

        #hashing the audio track using as keys it's signatures grouped in bands of size BAND_SIZE
        #storing the hashes in a list to retrieve the buckets to which the query belongs
        QueryBuckets = np.empty(0, dtype=int)
        for i in range(0, PERMUTATIONS, BAND_SIZE):
            Shingle = SignatureArray[i:(i+BAND_SIZE)]
            BucketKey = HashFunction(Shingle)
            QueryBuckets = np.append(QueryBuckets, BucketKey)

        #interection of buckets to which the query belongs
        # IntersectionElements = [*set.intersection(*[ShingleBuckets[key] for key in QueryBuckets[:3]])]
        TrackSimilarities = Counter()
        for q in QueryBuckets:
            TrackSimilarities.update(ShingleBuckets[str(q)])

        #track is...
        #print bucket elements to which the query belongs
        QueryName = AudioPath.split('.')[0]
        # for element in IDofTrack[TrackSimilarities.most_common()[0][0]]:
        track = IDofTrack[TrackSimilarities.most_common()[0][0]]
        #formatting
        Title = track.split('/')[-1][:-4].replace('_', ' ')
        Title = re.sub(r'[\d^-]', '', Title)
        Album = track.split('/')[1].split('_')
        Album = ' '.join([name.capitalize() for name in Album])
        Author = track.split('/')[0].split('_')
        Author = ' '.join([name.capitalize() for name in Author])
        QueryNameAlligned = str.center(AudioPath.split('.')[0] + ':', (13 + len(QueryName)))
        print(f'{QueryNameAlligned}{Author} - {Title} ({Album})')

        #most similar track has Jaccard similarity
        IDFirstSimilarity = TrackSimilarities.most_common()[0][0]
        IDSecondSimilarity = TrackSimilarities.most_common()[1][0]

        track = IDofTrack[TrackSimilarities.most_common()[1][0]]
        #formatting
        Title = track.split('/')[-1][:-4].replace('_', ' ')
        Title = re.sub(r'[\d^-]', '', Title)
        Album = track.split('/')[1].split('_')
        Album = ' '.join([name.capitalize() for name in Album])
        Author = track.split('/')[0].split('_')
        Author = ' '.join([name.capitalize() for name in Author])

        JaccardSimilarity = sum(SignatureMatrix[:, IDFirstSimilarity] == \
                                SignatureMatrix[:, IDSecondSimilarity]) / PERMUTATIONS
        print(f'{QueryName} best match: {Author} - {Title} ({Album}) with JS of {JaccardSimilarity: .3f}\n')

In the first example the parameers used are:
- `PERMUTATIONS` = 300
- `BAND_SIZE` = 3

and so:\
$b = 300 / 3 = 100$ \
$T = 0.215$

In [12]:
BAND_SIZE = 3
PERMUTATIONS = 300
ShingleBuckets = BucketingShingles()
EvaluateQuerys()

      track1:      Aerosmith - Dream On (Aerosmith)
track1 best match: Beatles - Baby s In Black (Beatles For Sale) with JS of  0.023

      track2:      Queen - I Want To Break Free (The Works)
track2 best match: Suzanne Vega - Small Blue Thing (Suzanne Vega) with JS of  0.030

      track3:      U2 - October (October)
track3 best match: Metallica - Bleeding Me (Load) with JS of  0.010

      track4:      Beatles - ObLaDi ObLaDa (The White Album Disc 1)
track4 best match: U2 - Love Is Blindness (Achtung Baby) with JS of  0.023

      track5:      Radiohead - Karma Police (Ok Computer)
track5 best match: Green Day - Dominated Love Slave (Kerplunk) with JS of  0.007

      track7:      Fleetwood Mac - Go Your Own Way (Rumours)
track7 best match: Green Day - Jinx (Nimrod) with JS of  0.017

      track8:      Green Day - American Idiot (American Idiot)
track8 best match: Radiohead - I Might Be Wrong (Amnesiac) with JS of  0.013

      track6:      Led Zeppelin - Heartbreaker (Led Zeppeli

Now the parameers are:
- `PERMUTATIONS` = 300
- `BAND_SIZE` = 10

and so:\
$b = 300 / 10 = 30$ \
$T = 0.712$

In [13]:
BAND_SIZE = 10
PERMUTATIONS = 300
ShingleBuckets = BucketingShingles()
EvaluateQuerys()

      track1:      Aerosmith - Dream On (Aerosmith)
track1 best match: Garth Brooks - I ve Got A Good Thing Going (Garth Brooks The Limited Series ) with JS of  0.023

      track2:      Queen - I Want To Break Free (The Works)
track2 best match: U2 - Running to Stand Still (The Joshua Tree) with JS of  0.027

      track3:      U2 - October (October)
track3 best match: Dave Matthews Band - Don t Drink the Water (Before These Crowded Streets) with JS of  0.000

      track4:      Beatles - ObLaDi ObLaDa (The White Album Disc 1)
track4 best match: Fleetwood Mac - Over My Head (Fleetwood Mac) with JS of  0.020

      track5:      Radiohead - Karma Police (Ok Computer)
track5 best match: Fleetwood Mac - Walk a Thin Line (Tusk) with JS of  0.013

      track7:      Fleetwood Mac - Go Your Own Way (Rumours)
track7 best match: Metallica - Metal Militia (Kill Em All) with JS of  0.017

      track8:      Green Day - American Idiot (American Idiot)
track8 best match: Cure - The Snakepit (Kiss 

Now the parameers are:
- `PERMUTATIONS` = 150
- `BAND_SIZE` = 5

and so:\
$b = 150 / 5 = 30$ \
$T = 0.507$

In [14]:
BAND_SIZE = 5
PERMUTATIONS = 150
SignatureMatrix = SignatureMatrix[:150]
ShingleBuckets = BucketingShingles()
EvaluateQuerys()

      track1:      Aerosmith - Dream On (Aerosmith)
track1 best match: Suzanne Vega - Song Of Sand (99 9 F ) with JS of  0.020

      track2:      Queen - I Want To Break Free (The Works)
track2 best match: Madonna - Bye Bye Baby (Erotica) with JS of  0.073

      track3:      U2 - October (October)
track3 best match: Green Day - King For A Day (Nimrod) with JS of  0.013

      track4:      Beatles - ObLaDi ObLaDa (The White Album Disc 1)
track4 best match: Radiohead - I Will (Hail To The Theif) with JS of  0.000

      track5:      Radiohead - Karma Police (Ok Computer)
track5 best match: Queen - Keep Passing The Open Windows (The Works) with JS of  0.013

      track7:      Fleetwood Mac - Go Your Own Way (Rumours)
track7 best match: Radiohead - A Wolf at the Door (Hail To The Theif) with JS of  0.027

      track8:      Green Day - American Idiot (American Idiot)
track8 best match: Cure - Bare (Wild Mood Swings) with JS of  0.013

      track6:      Led Zeppelin - Heartbreaker (Led 

Different combinations of the band size and the number of permutations gave different similarities of the querys but a significant variation of the $JS$ is not observed. \
The result could be imporved with:
- expanding the dataset, this would allow to have more shingles and more probability of similarities between them
- adding some "noise" to the peaks, de facto "spreading" the peaks over more array elements so to loosen the constrain that the peaks of the tracks should precisely match